In [18]:
!pip install -q groq

In [19]:
GROQ_KEY="Provide_API_KEY"

In [20]:
import json
import datetime
import os

from groq import Groq
date = datetime.datetime.now()
date
client = Groq(
    api_key=GROQ_KEY,
)
MODEL = 'llama3-70b-8192'

In [46]:
sys_prompt = """

You are a chatbot receptionists. Your role is to assist patients in scheduling appointments for our eye care services. Please respond appropriately and precisely.
You need these features. ask single question everytime and focus on that only
You can ask patient to share the details of the issue and also can get the information from the conversation
Must recomend specific clinic based on the issue and the clinics services if patient did not mentioned explicitly
don't recomend location as it has no role now
must need to check whether these email and cotnact are shared or not
Must ask for details for appointment like email and contact number for appointment before going further if not shared previously
saved the email and password and ask for confirmation
You can extract the time, period, and date of appointment from the patient conversation if shared otherwise you can ask as these information are necessary
You can Recomend doctor based doctor available slot and the patient appoitment information
You can also reask to reshdule it by sharing information about doctor availability
you can also provide alternative doctor od the same clinic to the patient if any mismatch or patient ask for alternative
you can only change patient information if the patient changed it otherwise stick to the conversation

**Instructions**
Extract the clinic details after greeting, this is compulsary for conversation
Respond in a precise manner using less words
try to understand the question and context before replying
respond like an assistance don't react like a doctor by asking details of the issue and symptom etc
don't go for out of the box responses
alway get the clinic details first before saving email and contact number
Always ensure the conversation is flowing correctly
verify the appoitment details once everything is done
Always check the information before confirming from your own
don't provide anything else from your custom knowledge other then context. this is unbearable in anycase


Please adhere to these steps to effectively assist patients. if any other question asked while following the instruction try to handle that too. Thank you!
"""

In [47]:
clinic_data = {
    "Karachi Medical Center": {
        "office_name": "Karachi Medical Center",
        "locations": [
            {"address": "123 Clifton Road, Karachi, Pakistan", "contact": "021-345-6789", "hours": "9 AM to 5 PM PKT Monday to Friday"},
            {"address": "456 Gulshan Avenue, Karachi, Pakistan", "contact": "021-987-6543", "hours": "9 AM to 3 PM PKT Saturday and Sunday"}
        ],
        "providers": [
            {"name": "Dr. Ahmed Ali", "bio": "Specializes in cardiology and internal medicine."},
            {"name": "Dr. Farah Khan", "bio": "Specializes in pediatric care and family medicine."}
        ],
        "services_offered": ["General Check-up", "Cardiology Consultation", "Pediatric Care", "Vaccinations"],
        "products_offered": ["Prescription Medication", "Health Supplements"],
        "parking_info": "Parking available at the clinic premises with no charges.",
        "visit_preparation": "Please bring a valid photo ID, your insurance card, and a list of any medications you are currently taking.",
        "warranty": "Prescription medication includes a one-year warranty against manufacturer defects.",
        "cancellation_policy": "Notify us at least 24 hours in advance. Failure to do so may result in a no-show fee.",
        "payment_types": ["Cash", "Visa Card", "MasterCard", "JazzCash"],
        "financing": "Yes, EMI option available with selected credit cards",
        "languages_spoken": ["Urdu", "English"],
        "accessibility": "Yes",
        "discounts": "Yes, 10% discount for senior citizens",
        "online_forms": "Yes",
        "online_payments": "Yes",
        "online_booking": "Yes",
        "arrival_instructions": "Patients are generally not required to arrive earlier than their scheduled appointment time.",
        "waitlist": "Yes"
    },
    "Lahore Health Clinic": {
        "office_name": "Lahore Health Clinic",
        "locations": [
            {"address": "789 Mall Road, Lahore, Pakistan", "contact": "042-123-4567", "hours": "8 AM to 4 PM PKT Monday to Friday"},
            {"address": "101 Liberty Market, Lahore, Pakistan", "contact": "042-765-4321", "hours": "8 AM to 2 PM PKT Saturday and Sunday"}
        ],
        "providers": [
            {"name": "Dr. Sana Malik", "bio": "Specializes in dermatology and cosmetic treatments."},
            {"name": "Dr. Imran Qureshi", "bio": "Specializes in orthopedic surgery."}
        ],
        "services_offered": ["Dermatology Consultation", "Cosmetic Treatments", "Orthopedic Surgery", "Physical Therapy"],
        "products_offered": ["Skin Care Products", "Orthopedic Supports"],
        "parking_info": "Parking available in the adjacent public parking lot.",
        "visit_preparation": "Please bring a valid photo ID, your insurance card, and a list of any medications you are currently taking.",
        "warranty": "Skin care products include a six-month warranty against manufacturer defects.",
        "cancellation_policy": "Notify us at least 24 hours in advance. Failure to do so may result in a no-show fee.",
        "payment_types": ["Cash", "Visa Card", "MasterCard", "Easypaisa"],
        "financing": "No",
        "languages_spoken": ["Urdu", "English", "Punjabi"],
        "accessibility": "No",
        "discounts": "Yes, 5% discount for students",
        "online_forms": "No",
        "online_payments": "No",
        "online_booking": "Yes",
        "arrival_instructions": "Please arrive 15 minutes before your scheduled appointment time.",
        "waitlist": "No"
    }
}


In [48]:
import datetime
import json

def greet_patient():
    return "just Greet the patient and ask to discuss issue"

def save_contact_info(email, phone_no):
  if email and phone_no:
    return f"Email: {email}, Phone no: {phone_no} saved successfully."
  else:
     return f' Did not shared email and password Clinic_detail'


def patient_date_time(date, time):
    return f"day: {date}, time: {time} saved successfully."


def confirm_appointment_details(confirmed):
    return "Appointment details confirmed." if confirmed else "Appointment details not confirmed. Please verify again."

def get_clinic_details(clinic_name=None):
    if clinic_name in clinic_data:
        return json.dumps(clinic_data[clinic_name], indent=4)
    else:
        return json.dumps(clinic_data, indent=4)

def get_doctor_details(doctor_name):
    for clinic, info in clinic_data.items():
        for provider in info.get("providers", []):
            if provider["name"].lower() == doctor_name.lower():
                return {
                    "clinic": clinic,
                    "doctor_name": provider["name"],
                    "bio": provider["bio"]
                }
    return "No Doctor Found"

def check_doctor_availability(clinic_name, doctor=None, day=None, date=None, time=None):
    available_doctors = {
        "Karachi Medical Center": {
            "Dr. Ahmed Ali": {
                "Monday": ["9 am", "11 am", "2 pm"],
                "Tuesday": ["10 am", "12 pm", "4 pm"],
                "Wednesday": ["9 am", "11 am", "3 pm"],
                "Thursday": ["10 am", "1 pm", "5 pm"],
                "Friday": ["9 am", "12 pm", "2 pm"]
            },
            "Dr. Farah Khan": {
                "Monday": ["8 am", "10 am", "1 pm"],
                "Tuesday": ["9 am", "11 am", "3 pm"],
                "Wednesday": ["8 am", "12 pm", "4 pm"],
                "Thursday": ["9 am", "11 am", "2 pm"],
                "Friday": ["8 am", "10 am", "1 pm"]
            }
        },
        "Lahore Health Clinic": {
            "Dr. Sana Malik": {
                "Monday": ["8 am", "10 am", "1 pm"],
                "Tuesday": ["9 am", "11 am", "3 pm"],
                "Wednesday": ["8 am", "12 pm", "4 pm"],
                "Thursday": ["9 am", "11 am", "2 pm"],
                "Friday": ["8 am", "10 am", "1 pm"]
            },
            "Dr. Imran Qureshi": {
                "Monday": ["9 am", "11 am", "2 pm"],
                "Tuesday": ["10 am", "12 pm", "4 pm"],
                "Wednesday": ["9 am", "11 am", "3 pm"],
                "Thursday": ["10 am", "1 pm", "5 pm"],
                "Friday": ["9 am", "12 pm", "2 pm"]
            }
        }
    }
    if not day:
        day = datetime.datetime.now().strftime("%A")
    day = day.title()
    if clinic_name in available_doctors and doctor in available_doctors[clinic_name]:
        if day in available_doctors[clinic_name][doctor]:
            if time:
                if time in available_doctors[clinic_name][doctor][day]:
                    return f"{doctor} is available at {clinic_name} on {day} at {time}."
                else:
                    return f"{doctor} is not available at {clinic_name} on {day} at {time}."
            elif date:
                datetime_str = f"{date} {day}"
                try:
                    target_date = datetime.datetime.strptime(datetime_str, "%Y-%m-%d %A")
                    if target_date.strftime("%A") == day:
                        return available_doctors[clinic_name][doctor][day]
                    else:
                        return f"No availability found for {doctor} at {clinic_name} on {date}."
                except ValueError:
                    return f"Invalid date format. Please provide date in YYYY-MM-DD format."
            else:
                return available_doctors[clinic_name][doctor][day]
        else:
            return f"{doctor} does not have scheduled availability at {clinic_name} on {day}."
    else:
        if clinic_name in available_doctors:
            return json.dumps(available_doctors[clinic_name], indent=4)
        else:
            return get_clinic_details()

def get_additional_clinic_details(clinic_name):
    additional_details = {
        "Karachi Medical Center": "Frame lines offered. Parking at Plaza East Garage with no charges.",
        "Lahore Health Clinic": "Contact lens products offered. Parking at Parking Garage at 5th and Golf Streets."
    }
    return additional_details.get(clinic_name, "Wrong clinic name")


def verify_appointment_details(email, phone_no, clinic_name, doctor, datetime, day):
    required_fields = {
        "email": email,
        "phone_no": phone_no,
        "clinic_name": clinic_name,
        "doctor": doctor,
        "datetime": datetime,
        "day": day
    }

    missing_fields = [field for field, value in required_fields.items() if not value]

    if missing_fields:
        return f"Missing required fields: {', '.join(missing_fields)}"
    else:
        return "All required fields present"

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "greet_patient",
            "description": "Respond to patient greet",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "save_contact_info",
            "description": "Save the patient email and phone number once the clinic is finalized",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {
                        "type": "string",
                        "description": "Email address of the patient",
                    },
                    "phone_no": {
                        "type": "string",
                        "description": "Phone number of the patient",
                    }
                },
                "required": ["email", "phone_no"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_doctor_details",
            "description": "get detail of the doctor",
            "parameters": {
                "type": "object",
                "properties": {
                    "doctor_name": {
                        "type": "string",
                        "description": "Name of the Doctor",
                    }
                },
                "required": ["doctor_name"]
            }
        }
    },
            {
        "type": "function",
        "function": {
            "name": "patient_date_time",
            "description": "Save the patient appointment date and time",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "description": "Patient appointment date",
                    },
                    "time": {
                        "type": "string",
                        "description": "Patient appoitment time",
                    }
                },
                "required": ["date", "time"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "confirm_appointment_details",
            "description": "Confirm the patient appoitment",
            "parameters": {
                "type": "object",
                "properties": {
                    "confirmed": {
                        "type": "boolean",
                        "description": "Whether the patient confirms the appointment details",
                    }
                },
                "required": ["confirmed"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "check_doctor_availability",
            "description": "Check the availability of a doctor",
            "parameters": {
                "type": "object",
                "properties": {
                    "clinic_name": {
                        "type": "string",
                        "description": "Name of the clinic",
                    },
                    "doctor": {
                        "type": "string",
                        "description": "Name of the doctor",
                    },
                    "day": {
                        "type": "string",
                        "description": "Optional day to check availability",
                    },
                    "date": {
                        "type": "string",
                        "description": "Optional date to check availability (YYYY-MM-DD)",
                    },
                    "time": {
                        "type": "string",
                        "description": "Optional time to check availability",
                    }
                }
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_clinic_details",
            "description": "check detailed information about the clinic",
            "parameters": {
                "type": "object",
                "properties": {
                    "clinic_name": {
                        "type": "string",
                        "description": "Name of the clinic",
                    }
                }
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_additional_clinic_details",
            "description": "check additional details about the clinic such as parking information.",
            "parameters": {
                "type": "object",
                "properties": {
                    "clinic_name": {
                        "type": "string",
                        "description": "Name of the clinic to get additional details for",
                    }
                },
                "required": ["clinic_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "verify_appointment_details",
            "description": "confirm the details of the appointment in the end including email, phone number, clinic name, doctor, datetime, and day.",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {
                        "type": "string",
                        "description": "Email address of the patient",
                    },
                    "phone_no": {
                        "type": "string",
                        "description": "Phone number of the patient",
                    },
                    "clinic_name": {
                        "type": "string",
                        "description": "Name of the clinic",
                    },
                    "doctor": {
                        "type": "string",
                        "description": "Name of the doctor",
                    },
                    "datetime": {
                        "type": "string",
                        "description": "Datetime of the appointment (YYYY-MM-DD HH:MM)",
                    },
                    "day": {
                        "type": "string",
                        "description": "Day of the appointment",
                    }
                },
                "required": ["email", "phone_no", "clinic_name", "doctor", "datetime", "day"]
            }
        }
    }
]


In [49]:
messages = [
    {
        "role": "system",
        "content": sys_prompt
    },
]
def run_conversation(user_prompt):
    messages.append({
        "role": "user",
        "content": user_prompt
    })
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=TOOLS,
        tool_choice="auto",
        max_tokens=4096,
        temperature=0,

    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    print(tool_calls)
    if tool_calls:
        available_functions = {
            "greet_patient":greet_patient,
            "save_contact_info": save_contact_info,
            "patient_date_time": patient_date_time,
            "check_doctor_availability": check_doctor_availability,
            "get_clinic_details":get_clinic_details,
            "get_additional_clinic_details":get_additional_clinic_details,
            "verify_appointment_details":verify_appointment_details,
        }

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions.get(function_name)

            if function_to_call:
                print(function_name)
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        # if messages:
        #     messages.pop()
        response_message = second_response.choices[0].message

    # else:
    messages.append({
        "role":"assistant",
        "content": response_message.content
    })
    return response_message.content
user_prompt = "hello"
print(run_conversation(user_prompt))

[ChatCompletionMessageToolCall(id='call_sg72', function=Function(arguments='{}', name='greet_patient'), type='function')]
greet_patient
Hello! Welcome to our eye care services. How can I assist you today? What seems to be the issue with your eyes that you'd like to discuss?


In [50]:
user_prompt = """Hello This is John Doe to book an appointment at the eyecare location. I am
experiencing some issues with my vision and would like to see a specialist as soon as possible.
Here are my details Name: John Doe DOB January 1, 1980 Preferred Appointment Date: June
10, 2024 Preferred Time: Morning, if possible Reason for Visit: Blurred vision and eye
discomfort Please call me back at your earliest convenience to confirm the appointment or
suggest an alternative time if my preferred slot is not available. You can reach me at the contact
number provided above. Thank you. Best regards John Doe"""
print(run_conversation(user_prompt))

[ChatCompletionMessageToolCall(id='call_det8', function=Function(arguments='{}', name='get_clinic_details'), type='function')]
get_clinic_details
Based on your issue with blurred vision and eye discomfort, I would recommend you visit our "Karachi Medical Center" clinic. They have a comprehensive range of services, including general check-ups and specialty care. 

Before we proceed, could you please share your email address and contact number so that I can assist you in scheduling an appointment?


In [51]:
user_prompt = "John@gmail.com 03219872721"
print(run_conversation(user_prompt))

[ChatCompletionMessageToolCall(id='call_v62r', function=Function(arguments='{"email":"John@gmail.com","phone_no":"03219872721"}', name='save_contact_info'), type='function')]
save_contact_info
I've saved your email address and contact number successfully. 

Now, let's schedule your appointment at the Karachi Medical Center. You mentioned your preferred appointment date is June 10, 2024, and preferred time is morning. I'll check the doctor's availability for you.


In [52]:
user_prompt = "please check and let me know"
print(run_conversation(user_prompt))

[ChatCompletionMessageToolCall(id='call_b22j', function=Function(arguments='{"clinic_name":"Karachi Medical Center","date":"2024-06-10","time":"morning"}', name='check_doctor_availability'), type='function')]
check_doctor_availability
Based on the doctor's availability, I found a suitable time slot for you at the Karachi Medical Center on June 10, 2024. Dr. Farah Khan has an available slot at 10 am. Would you like to book this appointment?


In [53]:
user_prompt = "yes"
print(run_conversation(user_prompt))

None
I've booked your appointment with Dr. Farah Khan at the Karachi Medical Center on June 10, 2024, at 10 am. 

Before we confirm, I just need to verify the appointment details with you: 

* Clinic: Karachi Medical Center
* Doctor: Dr. Farah Khan
* Date: June 10, 2024
* Time: 10 am
* Reason for Visit: Blurred vision and eye discomfort
* Email: John@gmail.com
* Phone no: 03219872721

Is everything correct?


In [54]:
user_prompt = "yes correct"
print(run_conversation(user_prompt))

None
Your appointment has been confirmed. You will receive a confirmation call from our team shortly. 

Please arrive 15 minutes prior to your scheduled appointment time and bring a valid photo ID, your insurance card, and a list of any medications you are currently taking.

If you need to reschedule or have any questions, feel free to reach out to us at the contact number provided.

Thank you for choosing our eye care services. We look forward to assisting you on June 10, 2024.
